In [ ]:
#load libraries

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import os
import copy
import seaborn as sns
import importlib
from scipy.signal import argrelmax, find_peaks, peak_widths
import scanpy as sc
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale, minmax_scale
from sklearn.metrics import silhouette_score
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['mathtext.it'] = 'Arial:italic'
mpl.rcParams['mathtext.rm'] = 'Arial'
codedir = os.getcwd()
#load cmif libraries
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cmIF')
from mplex_image import visualize as viz, process, preprocess, normalize

In [ ]:
os.chdir(codedir)

In [ ]:
np.random.seed(222)

# Table of contents <a name="contents"></a>
1. [Load Data](#load)
2. [Normalize](#norm)
3. [Visualize Normalization](#normviz)
4. [leiden for cell typing](#clusterlei)
5. [Leiden cluster](#clust1)


note:

    Could you make composite fraction bar graph only  in following regions?

    Bx2: SMTBx2-5-Scene-001_ROI1-2000-9000-2500-2500
    Bx3: SMTBx3-Scene-004_ROI2-20900-15494-2500-2500
    Bx4: HTA-33-Scene-002_ROI1-3271-607-2500-2500

    If we can have it in Bx1
    Bx: SMTBx1-Scene-003_ROI1-2440-220-2500-2500



In [ ]:
#load data
os.chdir(f'{codedir}/paper_data')

In [ ]:
s_date = '20210402'
if not os.path.exists(s_date):
    os.mkdir(s_date)

# Load Data <a name="load"></a>

[contents](#contents)

In [ ]:
os.chdir(f'{codedir}/paper_data')
df_file = pd.DataFrame(index=os.listdir())
df_file = df_file[df_file.index.str.contains('FilteredMeanIntensity_DAPI')]
df_file['tissue'] = [item.split('_')[1] for item in df_file.index]
df_file['dapi'] = ['DAPI' + item.split('y_DAPI')[1].split('.')[0] for item in df_file.index]
ls_sample = df_file.tissue.tolist()
d_dapi = dict(zip(df_file.tissue.tolist(),df_file.dapi.tolist()))
d_dapi.update({'JE-TMA-60': 'DAPI10_DAPI2'})
df_mi = pd.DataFrame()
df_xy = pd.DataFrame()
df_edge = pd.DataFrame()

for s_sample in sorted(set(ls_sample)):
    #if not s_sample.find('HTA')>-1:
        print(f'loading {s_sample}')
        df_mi = df_mi.append(pd.read_csv(f'{codedir}/paper_data/features_{s_sample}_FilteredMeanIntensity_{d_dapi[s_sample]}.csv', index_col=0))
        df_xy = df_xy.append(pd.read_csv(f'{codedir}/paper_data/features_{s_sample}_CentroidXY.csv',index_col=0))
        if os.path.exists(f'{codedir}/paper_data/features_{s_sample}_EdgeCells153pixels_CentroidXY.csv'):
            df_edge = df_edge.append(pd.read_csv(f'{codedir}/paper_data/features_{s_sample}_EdgeCells153pixels_CentroidXY.csv',index_col=0))

In [ ]:
#sorted(df_mi.columns[df_mi[~df_mi.index.str.contains('JE-TMA-60')].isna().sum() != 0])

In [ ]:
ls_marker = ['AR_nuclei', 'CD20_perinuc5', 'CD31_perinuc5', 'CD3_perinuc5',  'CD44_perinuc5', 'CD45_perinuc5',#'CD44_nucadj2',
 'CD4_perinuc5', 'CD68_perinuc5','CD8_perinuc5', 'CK14_cytoplasm', 'CK17_cytoplasm', 'CK19_cytoplasm', 'CK5_cytoplasm',
 'CK7_cytoplasm', 'CK8_cytoplasm', 'ColI_perinuc5', 'ColIV_perinuc5','CoxIV_perinuc5','EGFR_cytoplasm', 'ER_nuclei',
 'Ecad_cytoplasm', 'FoxP3_nuclei', 'GRNZB_nuclei', 'H3K27_nuclei','H3K4_nuclei', 'HER2_cellmem25','Ki67_nuclei',
 'LamAC_nuclei', 'PCNA_nuclei', 'PD1_perinuc5', 'PDPN_perinuc5','DAPI2_nuclei',  # 'ER_nuclei25','HER2_cytoplasm','PgR_nuclei','Vim_nucadj2'
 'Vim_perinuc5', 'aSMA_perinuc5', 'pHH3_nuclei', 'pRB_nuclei', 'pS6RP_perinuc5','slide_scene',
           ] # CD8R bad, 'gH2AX_nuclei' in R11 Bx3 not included

df_mi = df_mi.loc[:,ls_marker]
 
# old 
#df_mi = df_mi.loc[:,['HER2_cellmem25', 'DAPI2_nuclei',# 'CD44_nucadj2', 'Vim_nucadj2','ER_nuclei25','HER2_cytoplasm',
#       'CD20_perinuc5', 'CD3_perinuc5', 'CD31_perinuc5', 'CD4_perinuc5','CD44_perinuc5', 'CD45_perinuc5', 'CD68_perinuc5', 'CD8_perinuc5',
#       'PD1_perinuc5', 'PDPN_perinuc5', 'Vim_perinuc5', 'aSMA_perinuc5','CK14_cytoplasm', 'CK17_cytoplasm', 'CK19_cytoplasm', 'CK5_cytoplasm',
#       'CK7_cytoplasm', 'Ecad_cytoplasm', 'ER_nuclei', 'Ki67_nuclei', 'LamAC_nuclei','PCNA_nuclei', 'pHH3_nuclei', 'slide_scene']]


df_mi['batch'] = [item.split('_')[0] for item in df_mi.index]
#df_mi['scene'] = [item.split('_')[1] for item in df_mi.index]

## Deal with JE-TMA-60

In [ ]:
# markers in JE-TMA-60
#'JE-TMA-60_scene06', 'JE-TMA-60_scene08', 'JE-TMA-60_scene09', 'JE-TMA-60_scene10', 'JE-TMA-60_scene11', 'JE-TMA-60_scene13'
# R5 is CK17.PDPN.CD45.FoxP3

In [ ]:
df_R5 = pd.read_csv(f'{codedir}/paper_data/features_JE-TMA-60_FilteredMeanIntensity_DAPI5_DAPI2.csv',index_col=0)
df_R4 = pd.read_csv(f'{codedir}/paper_data/features_JE-TMA-60_FilteredMeanIntensity_DAPI4_DAPI2.csv',index_col=0)
df_R10 = df_mi[df_mi.batch=='JE-TMA-60']

In [ ]:
ls_scene = set(df_R10.slide_scene)

In [ ]:
ls_na = set([item.split('_cell')[0] for item in df_R5.index]) - set([item.split('_cell')[0] for item in df_R10.index])

In [ ]:
#slect markers, scenes for normalization (based on JE-TMA-60 tissue loss)
ls_pos = ['HER2_cellmem25','CK19_cytoplasm','CK7_cytoplasm','CK8_cytoplasm','Ecad_cytoplasm','ER_nuclei','Ki67_nuclei','LamAC_nuclei',
          'PCNA_nuclei','pHH3_nuclei','Vim_perinuc5','DAPI2_nuclei','H3K27_nuclei','H3K4_nuclei', 'pRB_nuclei','pS6RP_perinuc5',
         'CoxIV_perinuc5','EGFR_cytoplasm']
ls_R5 = ['CK17_cytoplasm','PDPN_perinuc5','CD45_perinuc5','FoxP3_nuclei'] #
ls_R4 = ['pHH3_nuclei','CK14_cytoplasm','Ki67_nuclei','CK19_cytoplasm','CK5_cytoplasm','HER2_cellmem25',
        'Ecad_cytoplasm', 'ER_nuclei','CD44_perinuc5', 'PCNA_nuclei','aSMA_perinuc5','CD3_perinuc5','EGFR_cytoplasm']
ls_bad = ['CD20_perinuc5', 'CD31_perinuc5', 'CD4_perinuc5', 'CD68_perinuc5', 'CD8_perinuc5','PD1_perinuc5',
         'ColI_perinuc5', 'ColIV_perinuc5'] #'CK7_cytoplasm', #'LamAC_nuclei',
#ls_good = ['CK7_cytoplasm','Vim_perinuc5','LamAC_nuclei']

#R4
df =  df_mi[df_mi.batch!='JE-TMA-60']
df = df.append(df_R4.loc[:,ls_R4])
#R5
ls_index = df_R5.loc[df_R5.index.isin(df_R4.index)].index
df.loc[ls_index,ls_R5] = df_R5.loc[ls_index,ls_R5]

#fill R6-8
ls_index = df_mi.loc[(df_mi.slide_scene.isin(ls_scene)) & (df_mi.index.isin(df_R4.index))].index
df.loc[ls_index,ls_pos] = df_R10.loc[ls_index,ls_pos]

#
df['batch'] = [item.split('_')[0] for item in df.index]
#df['scene'] = [item.split('_')[1] for item in df.index]
df['slide_scene'] = [item.split('_cell')[0] for item in df.index]

## filter edge cells

In [ ]:
#filter out unwanted cells
d_filter = {#41 (not used)
            'JE-TMA-41_scene01':(df_xy.DAPI_Y > 5000),'JE-TMA-41_scene03':(df_xy.DAPI_Y > 5000),
            'JE-TMA-41_scene04':(df_xy.DAPI_Y < 1500),'JE-TMA-41_scene05':(df_xy.DAPI_Y > 5000),
            'JE-TMA-41_scene06':(df_xy.DAPI_Y < 1500),'JE-TMA-41_scene08':(df_xy.DAPI_Y < 1500),
            'JE-TMA-41_scene09':(df_xy.DAPI_Y > 5000),'JE-TMA-41_scene11':(df_xy.DAPI_Y < 1500),
            #43
            'JE-TMA-43_scene09':(df_xy.DAPI_Y < 1200),'JE-TMA-43_scene14':(df_xy.DAPI_Y < 1200),
            #60
            'JE-TMA-60_scene02':(df_xy.DAPI_X < 1500),'JE-TMA-60_scene05':(df_xy.DAPI_X < 1500),
            'JE-TMA-60_scene11':(df_xy.DAPI_Y < 1500),'JE-TMA-60_scene14':(df_xy.DAPI_X < 1500),
            'JE-TMA-60_scene06':(df_xy.DAPI_Y < 1500),'JE-TMA-60_scene08':(df_xy.DAPI_Y > 5000),
            'JE-TMA-60_scene10':(df_xy.DAPI_Y < 1500),
            #63
            'JE-TMA-62_scene01':(df_xy.DAPI_Y > 5000),
            'JE-TMA-62_scene02':(df_xy.DAPI_X > 5000),'JE-TMA-62_scene03':(df_xy.DAPI_X < 1000),
            'JE-TMA-62_scene04':(df_xy.DAPI_Y < 1500),'JE-TMA-62_scene06':(df_xy.DAPI_X < 1000),
            'JE-TMA-62_scene08':(df_xy.DAPI_Y > 5000),'JE-TMA-62_scene10':(df_xy.DAPI_Y < 1500),
            #'SMTBx1-16_scene001':(df_xy.DAPI_Y > 1), #keep scene 1 for manual thresholding
            'SMTBx2-3_scene002':(df_xy.DAPI_Y > 5000),'SMTBx3_scene004':(df_xy.DAPI_X <11000),
            'SMTBx3_scene005':(df_xy.DAPI_X > 0),'SMTBx4-3_scene001':(df_xy.DAPI_Y < 2400),
            'SMTBx2-5_scene002':(df_xy.DAPI_Y > 5000),'HTA-33_scene003':(df_xy.DAPI_Y > 9000)}
d_filter2 = {'JE-TMA-60_scene02':(df_xy.DAPI_Y > 4500)}
ls_filter_all = []
for s_scene, filtercon in d_filter.items():
    ls_filter = df_xy[(df_xy.slide_scene==s_scene) & filtercon].index.tolist()
    ls_filter_all = ls_filter_all + ls_filter
for s_scene, filtercon in d_filter2.items():
    ls_filter = df_xy[(df_xy.slide_scene==s_scene) & filtercon].index.tolist()
    ls_filter_all = ls_filter_all + ls_filter
#filter edge
ls_filter_all = ls_filter_all + df_edge.index.tolist()
df_filter_mi = df[(~df.index.isin(ls_filter_all))]


In [ ]:
df_cluster = df_filter_mi.loc[:,['HER2_cellmem25','slide_scene']]
df_cluster['cluster'] = 1
df_cluster.drop('HER2_cellmem25',axis=1,inplace=True)
import importlib
importlib.reload(viz)
%matplotlib inline
viz.plot_clusters(df_cluster,df_xy,s_num='few')

In [ ]:
#match controls to biopsies
import warnings
warnings.filterwarnings('ignore')
d_replace = {'BC44290-146': 'JE-TMA-41',
 'SMTBx2-3': 'JE-TMA-41',
 'SMTBx2-5':'JE-TMA-43',
 'SMTBx3':'JE-TMA-60',
 'SMTBx4-3':'JE-TMA-62'}
df_filter_mi.loc[:,'batch'] = df_filter_mi.batch.replace(d_replace)

In [ ]:
#standardize the scenes
d_replace = { 'JE-TMA-41_scene13':'JE-TMA-41_scene14',
             'JE-TMA-41_scene12':'JE-TMA-41_scene13',
             'JE-TMA-62_scene13':'JE-TMA-62_scene14',
             'JE-TMA-62_scene12':'JE-TMA-62_scene13'}
df_filter_mi.loc[:,'scene'] = df_filter_mi.slide_scene.replace(d_replace)

In [ ]:
df_filter_mi.merge(df_xy.loc[:,['DAPI_X', 'DAPI_Y', 'nuclei_area', 'nuclei_eccentricity']],left_index=True,right_index=True)

In [ ]:
df_out = df_filter_mi.merge(df_xy.loc[:,['DAPI_X', 'DAPI_Y', 'nuclei_area', 'nuclei_eccentricity']],left_index=True,right_index=True)
len(df_out)

In [ ]:
#2-23 contains NAs
#2-22 the NAs were filled with random gaussian data
# 0302 include scene 1 Bx1
# 0318 just Bx2 - 4, (Bx2-5)
# 20210324 has HTA9-1-33
if not os.path.exists('20210324_SMTBx1-4_JE-TMA-43_60_62_FilteredMeanIntensity.csv'):
    print('saving csv')
    #df_out.to_csv('20210223_SMTBx1-4_JE-TMA-41_60_62_BC44290-146.csv')
    df_out.to_csv('20210324_SMTBx1-4_JE-TMA-43_60_62_FilteredMeanIntensity.csv') 

In [ ]:
#2-23 contains NAs
#2-22 the NAs were filled with random gaussian data
# 0302 include scene 1 Bx1
# 0318 just Bx2 - 4, (Bx2-5)
if not os.path.exists('20210320_SMTBx2-4_JE-TMA-43_60_62_FilteredMeanIntensity.csv'):
    print('saving csv')
    #df_out.to_csv('20210223_SMTBx1-4_JE-TMA-41_60_62_BC44290-146.csv')
    df_out.to_csv('20210320_SMTBx2-4_JE-TMA-43_60_62_FilteredMeanIntensity.csv') 

## Normalization <a name="norm"></a>

use ComBat.

[contents](#contents)

In [ ]:
df_mi = pd.read_csv('20210320_SMTBx2-4_JE-TMA-43_60_62_FilteredMeanIntensity.csv',index_col=0)

In [ ]:
df_mi.scene.unique()

In [ ]:
ls_pos = ['HER2_cellmem25','CK19_cytoplasm','CK7_cytoplasm','CK8_cytoplasm','Ecad_cytoplasm','ER_nuclei','Ki67_nuclei','LamAC_nuclei',
          'PCNA_nuclei','pHH3_nuclei','Vim_perinuc5','DAPI2_nuclei','H3K27_nuclei','H3K4_nuclei', 'pRB_nuclei','pS6RP_perinuc5',
         'CoxIV_perinuc5','EGFR_cytoplasm']
ls_R5 = ['CK17_cytoplasm','PDPN_perinuc5','CD45_perinuc5','FoxP3_nuclei'] #
ls_R4 = ['pHH3_nuclei','CK14_cytoplasm','Ki67_nuclei','CK19_cytoplasm','CK5_cytoplasm','HER2_cellmem25',
        'Ecad_cytoplasm', 'ER_nuclei','CD44_perinuc5', 'PCNA_nuclei','aSMA_perinuc5','CD3_perinuc5','EGFR_cytoplasm']
ls_bad = ['CD20_perinuc5', 'CD31_perinuc5', 'CD4_perinuc5', 'CD68_perinuc5', 'CD8_perinuc5','PD1_perinuc5',
         'ColI_perinuc5', 'ColIV_perinuc5']

In [ ]:
#select normalization scenes
ls_R10_scene = ['scene06', 'scene08', 'scene09', 'scene10', 'scene11', 'scene13']
ls_R10 = ['HER2_cellmem25', 'CK19_cytoplasm', 'CK7_cytoplasm', 'Ecad_cytoplasm', 'ER_nuclei', 'Ki67_nuclei', 'LamAC_nuclei',
          'PCNA_nuclei','pHH3_nuclei', 'Vim_perinuc5','CD44_perinuc5','DAPI2_nuclei', #adding following:
          'CK8_cytoplasm','CoxIV_perinuc5', 'EGFR_cytoplasm', 'H3K27_nuclei', 'H3K4_nuclei', 'pRB_nuclei', 'pS6RP_perinuc5']
#note: CK17 may have quenching artifact; PDPN not good in Bx1, so just CD45 important
#'CK17_cytoplasm','PDPN_perinuc5', #'FoxP3_nuclei' not in full set
ls_R5 = ['PDPN_perinuc5','CD45_perinuc5','FoxP3_nuclei', 'aSMA_perinuc5','CD3_perinuc5']  # aSMA because N breast, scene 01 better than 07 for immune
ls_R5_scene = ['scene01','scene03','scene04']
#old ls_R4 = ['pHH3_nuclei','CK14_cytoplasm','Ki67_nuclei','CK19_cytoplasm','CK5_cytoplasm','HER2_cellmem25',
#        'Ecad_cytoplasm', 'ER_nuclei','CD44_perinuc5', 'PCNA_nuclei','aSMA_perinuc5','CD3_perinuc5','DAPI2_nuclei']
#can scene 7 be good control for CD3 and CK14 and CK5?, yes. R1 doen't add much
ls_R4 = [ 'CK14_cytoplasm', 'CK5_cytoplasm','CK17_cytoplasm'] #'CD3_perinuc5',
ls_R4_scene = ['scene02','scene07']
ls_bad = ['CD20_perinuc5', 'CD31_perinuc5', 'CD4_perinuc5', 'CD68_perinuc5', 'CD8_perinuc5','PD1_perinuc5']

In [ ]:
set(df_mi.batch)
#df_mi = df_mi.loc[df_mi.batch!='JE-TMA-60']
df_mi['slide_scene'] = df_mi.scene
df_mi['scene'] = [item.split('_')[1] for item in df_mi.slide_scene]

In [ ]:
#dropped 60
df_norm_all=pd.DataFrame(index=df_mi.dropna().index)

#not dropped 60
df_norm_all=pd.DataFrame(index=df_mi.index)

In [ ]:
#1 fit on scenes that are good through round 10 and markers that are positive on those scenes "pos"
for s_type in ['R4','R5','R10']:
    if s_type == 'R10':
        ls_pos = ls_R10
        ls_scene = ls_R10_scene

    #2 fit on scenes that are good until R4, and R1-4 markers
    if s_type == 'R4':
        ls_pos = ls_R4
        ls_scene = ls_R4_scene # + ls_R5_scene + ls_R10_scene 

    #3 fit on scene that are good until R5, and R5 markers
    if s_type == 'R5':
        ls_pos = ls_R5
        ls_scene = ls_R5_scene

    #fit
    b_control = ((df_mi.index.str.contains('JE-TMA')) & (df_mi.scene.isin(ls_scene)) & (df_mi.loc[:,ls_pos].isna().sum(axis=1)==0))
    data = df_mi.loc[b_control,ls_pos].T
    batch = df_mi.loc[b_control,'batch']
    gamma_star, delta_star, stand_mean, var_pooled = normalize.combat_fit(data, batch)
    #transform
    #data = df_mi.loc[df_mi.batch!='SMTBx1-16',df_mi.dtypes=='float64'].drop(['DAPI_X','DAPI_Y'],axis=1).T
    data = df_mi.loc[df_mi.batch!='SMTBx1-16',ls_pos].T
    batch = df_mi.loc[df_mi.batch!='SMTBx1-16','batch']
    bayesdata = normalize.combat_transform(data,batch,gamma_star,delta_star,stand_mean, var_pooled)
    df_norm = bayesdata.T
    df_norm_all = df_norm_all.merge(df_norm,left_index=True,right_index=True,how='left')

In [ ]:
df_norm_all.tail()

In [ ]:
# run after #1, 2 and 3
df_norm_all =  df_norm_all.merge(df_mi.loc[:,['batch','DAPI_X','DAPI_Y','scene','nuclei_area','nuclei_eccentricity']],left_index=True,right_index=True)

In [ ]:
#old check
df_norm = df_norm.merge(df_mi.loc[:,['batch','DAPI_X','DAPI_Y','scene','nuclei_area','nuclei_eccentricity']],left_index=True,right_index=True)
#df_mi.loc[b_control,:].drop(['DAPI_X','DAPI_Y'],axis=1).groupby('batch').mean()
#df_mi[df_mi.index.str.contains('JE-TMA')].drop(['DAPI_X','DAPI_Y'],axis=1).groupby('batch').std()
#check
df_norm.loc[b_control,:].drop(['DAPI_X','DAPI_Y'],axis=1).groupby('batch').mean()
#df_norm[df_norm.index.str.contains('JE-TMA')].drop(['DAPI_X','DAPI_Y'],axis=1).groupby('batch').std()

In [ ]:
#df_norm_all.to_csv('20210320_SMTBx2-4_JE-TMA-43_60_62_normalized.csv')
#df_norm_all.to_csv('20210325_SMTBx2-4_JE-TMA-43_60_62_normalized.csv')

## Umap Visualize Normalization <a name="normviz"></a>

[contents](#contents)

In [ ]:
#s_sample = '20210320_SMTBx2-4_JE-TMA-43_60_62'
s_sample = '20210325_SMTBx2-4_JE-TMA-43_60_62'
df_norm_all = pd.read_csv(f'{s_sample}_normalized.csv',index_col=0)
df_norm_all.rename({'nuclei_area':'area','nuclei_eccentricity':'eccentricity','DAPI_X':'DAPIX',
       'DAPI_Y':"DAPIY"},axis=1, inplace=True)
df_norm_all.columns = [item.split('_')[0] for item in df_norm_all.columns]
df_norm_all['slide'] = [item.split('_')[0] for item in df_norm_all.index]
df_norm_all['scene'] = [item.split('_')[1] for item in df_norm_all.index]
df_norm_all['slide_scene'] = [item.split('_cell')[0] for item in df_norm_all.index]

In [ ]:
df_norm_all = df_norm_all.loc[~df_norm_all.slide_scene.isin(['JE-TMA-43_scene01','JE-TMA-62_scene01'])]

In [ ]:
# visualize
%matplotlib inline
s_type = 'w-60_no01'
#adata = sc.AnnData(df_norm_all.loc[:,df_norm_all.dtypes=='float64'].drop(['DAPIX','DAPIY'],axis=1)) 
ls_drop = ['DAPIX','DAPIY','DAPI2','LamAC','pHH3','FoxP3','CoxIV',
          'H3K27','H3K4','pRB','pS6RP','aSMA','PDPN'] #aSMA, PDPN not well norm
adata = sc.AnnData(df_norm_all.dropna().loc[:,df_norm_all.dtypes=='float64'].drop(ls_drop,axis=1))
adata.obs['batch'] = df_norm_all.dropna().loc[:,'batch']
adata.obs['scene'] =  df_norm_all.dropna().loc[:,'scene'].replace({'scene001':'Bx', 'scene002':'Bx','scene003':'Bx', 'scene004':'Bx', 'scene005':'Bx'})
adata.obs['tissue'] = df_norm_all.dropna().loc[:,'slide']
# reduce dimensionality (PCA)
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
#sc.pl.pca(adata)
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# calculate neighbors
n_neighbors = 31
n_pcs=len(adata.var.index) - 1
results_file =  f'{s_sample}_{n_neighbors}neighbors_{n_pcs}pcs_{len(adata.var.index)}markers.h5ad'

In [ ]:
results_file

In [ ]:
d_celline = {'scene02':'HCC1143',
 'scene03':'HCC3153',
 'scene04':'N.Breast',
 'scene05':'T47D',
 'scene06':'T47D',
 'scene07':'Tonsil',
 'scene08':'BT474',
 'scene09':'BT474',
 'scene10':'AU565',
 'scene11':'AU565',
 'scene12':'MDAMB436',
 'scene13':'MDAMB436',
 'scene14':'MDAMB436'}



In [ ]:

# calculate neighbors
if os.path.exists(results_file):
    adata = sc.read_h5ad(results_file)
    print('loading umap')
else:
    # calculate neighbors 
    print('calculating umap')
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)
    sc.tl.umap(adata)
    #save results
    if not os.path.exists(results_file):
        adata.write(results_file)

# umap plus scenes
fig,ax = plt.subplots(figsize=(3,2.5),dpi=600)
figname = f'UmapScene_{s_type}_{n_pcs+1}markers.png'
sc.pl.umap(adata, color='scene',save=figname,title=f'TMA Core',ax=ax)


# umap plus tissue
fig,ax = plt.subplots(figsize=(3,2.5),dpi=600)
figname = f'UmapTissue_{s_type}_{n_pcs+1}markers.png'
adata.obs['Tissue'] = adata.obs['tissue'].replace({'SMTBx2-5':'Bx2', 'SMTBx3':'Bx3','SMTBx4-3':'Bx4'})
sc.pl.umap(adata, color='Tissue',save=figname,title=f'Tissue',ax=ax)


# umap plus cell line
adata.obs['Subtype'] = adata.obs.scene.replace(d_celline)
fig,ax = plt.subplots(figsize=(3,2.5),dpi=600)
figname = f'UmapSubtype_{s_type}_{n_pcs+1}markers.png'
sc.pl.umap(adata, color='Subtype',save=figname,title=f'Subtype',ax=ax)


#umap plot
ls_marker = adata.var.index.tolist()
figname = f"Umap_{s_type}_{n_pcs+1}markers.png"
axes = sc.pl.umap(adata, color=ls_marker,wspace=.25,save=figname,vmin='p1.5',vmax='p98.5',ncols=3,show=False)


In [ ]:
#umap plot
ls_marker = adata.var.index.tolist()
figname = f"Umap_{s_type}_{n_pcs+1}markers.png"
fig = sc.pl.umap(adata, color=ls_marker,wspace=.25,vmin='p1.5',vmax='p98.5',ncols=3,show=False,return_fig=True)
ax_list = fig.axes
for ax in ax_list:
    ax.set_title(ax.get_title(),fontsize=28)
fig.savefig(f'figures/{figname}',dpi=600)


## cluster leiden <a name="clusterlei"></a>

[contents](#contents)

cluster on the markers that are normalized well

In [ ]:
resolution = 0.45
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_pcs}pcs_{len(adata.var.index)}markers_leiden{resolution}.h5ad'
#save
if not os.path.exists(results_file):
    sc.tl.leiden(adata,resolution=resolution)
else:
    adata = sc.read_h5ad(results_file)
    print('loading leiden')    
fig,ax = plt.subplots(figsize=(3,2.5),dpi=600)
figname=f'leiden_{resolution}.png'
sc.pl.umap(adata, color='leiden',ax=ax,save=figname)
#seaborn clustermap
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
g = sns.clustermap(df_p.groupby('leiden').mean(),z_score=1,figsize=(4,4),cmap='viridis',
                   vmin=-1.5,vmax=1.5) 
#g.savefig(f'./figures/clustermap_leiden.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
#scanpy matrixplot
fig,ax = plt.subplots(figsize=(5,5), dpi=200)
figname=f'Matrixplot_leiden_{resolution}.png'
sc.pl.matrixplot(adata, var_names=marker_genes, groupby=f'leiden',title='',categories_order=categories_order,
                 ax=ax,save=figname,standard_scale='var',colorbar_title='Relative\nintensity',
                #var_group_positions=[(3,23),(24,31),(32,42),(43,51)],
                 #var_group_labels=['tumor','T-cell','muscle\n +AF','immune\n+stroma'],
                #var_group_rotation=0
                )

#save
if not os.path.exists(results_file):
    adata.write(results_file)

# Leiden barplots <a name="clust1"></a>


[contents](#contents)

In [ ]:
ls_order = [
       'Bx2','Bx3','Bx4',#'JE-TMA-43_scene01','JE-TMA-62_scene01',
       'JE-TMA-43_scene02', 'JE-TMA-62_scene02',
       'JE-TMA-43_scene03', 'JE-TMA-62_scene03', 'JE-TMA-43_scene04',
       'JE-TMA-62_scene04', 'JE-TMA-43_scene05', 'JE-TMA-62_scene05',
       'JE-TMA-43_scene06','JE-TMA-60_scene06', 'JE-TMA-62_scene06', 'JE-TMA-43_scene07',
       'JE-TMA-62_scene07', 'JE-TMA-43_scene08','JE-TMA-60_scene08', 'JE-TMA-62_scene08',
       'JE-TMA-43_scene09','JE-TMA-60_scene09', 'JE-TMA-62_scene09','JE-TMA-43_scene10', 'JE-TMA-62_scene10','JE-TMA-60_scene10',
       'JE-TMA-43_scene11', 'JE-TMA-60_scene11', 'JE-TMA-62_scene11', 'JE-TMA-43_scene13',
       'JE-TMA-62_scene12', 'JE-TMA-43_scene14','JE-TMA-60_scene13', 'JE-TMA-62_scene13']      

In [ ]:
ls_order_r = ls_order[::-1]

In [ ]:
#load original
'''
s_sample = '20210320_SMTBx2-4_JE-TMA-43_60_62'
n_neighbors = 30
n_pcs = 19
n_markers = n_pcs+1
resolution = 0.5
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_pcs}pcs_{n_markers}markers_leiden{resolution}.h5ad'
adata1 = sc.read_h5ad(results_file) 

d_cluster = {'14': '14: Basal',
'5': '5: T cell',
'12': '12: T cell',
'10': '10: Myoepithelial',
'1': '1: Mesenchymal',
'16': '16: Prolif.',
'15': '15: Vim+ FB (Bx3)',
'11': '11: Vim+ FB (Bx4)',
'13': '13: Vim+ FB (Bx2)',
'7': '7: HER2++',
'9': '9: EGFR+ Basal',
'3': '3: HER2+',
'8': '8: HER2++, Ecad-',
'0': '0: ER+ (Bx4)',
'2': '2: ER+, PCNA+ ',
'4': '4: ER+, EGFR+ (Bx3)',
'6': '6: ER+ (Bx2)'}
d_clust_names = dict(zip([item[0] for item in d_cluster.items()],[item[1].split(': ')[1] for item in d_cluster.items()]))
'''

In [ ]:
#load
s_sample = '20210325_SMTBx2-4_JE-TMA-43_60_62'
n_neighbors = 31
n_pcs = 17
n_markers = n_pcs+1
resolution = 0.45
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_pcs}pcs_{n_markers}markers_leiden{resolution}.h5ad'
adata1 = sc.read_h5ad(results_file) 
print(results_file)

In [ ]:
if resolution == 0.5:
    d_cluster = {'14': '14: Basal','12': '12: T cell','16': '16: Prolif.','7': '7: ER+ (Bx2)','13': '13: Luminal (N.Breast)',
    '1': '1: ER+ PCNA+ (T47D)','0': '0: ER+ (Bx4)','15': '15: ER+ CK8++ (Bx4)','4': '4: ER+, EGFR+ (Bx3)','18': '18: ER+, EGFR+ (Bx3)',
    '17': '17: (Bx3)','10': '10: FB (Bx4)','11': '11: FB (Bx2)','3': '3: CD44+','9': '9: CD44+', '8': '8: EGFR+ Basal',
    '5': '5: HER2++','6': '6: HER2+','2': '2: HER2++, Ecad-',}
elif resolution == 0.45:
    d_cluster = {'15':'15: Basal',
                 '12':'12: T cell',
                 '16': '16: prolif.',
                 '5':'5: ER+, EGFR+ (Bx3)',
                '0':'0: ER+ (Bx4)',
                '1':'1: ER+, PCNA+',
                '7':'7: ER- (Bx2)',
                '9':'9: ER+ (Bx2)',
                '8':'8: EGFR+ Basal',
                '4':'4: HER2+',
                '3':'3: HER2+',
                '6':'6: HER2+, Ecad-',
                '2':'2: Mesenchymal',
                '10':'10: Mesenchymal',
                '14':'14: fibroblast',
                '11':'11: fibroblast',
                '13':'13: fibroblast'}
d_clust_names = dict(zip([item[0] for item in d_cluster.items()],[item[1].split(': ')[1] for item in d_cluster.items()]))

In [ ]:
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
#sns.set(font_scale=1.19)
#seaborn clustermap
df_p = pd.DataFrame(data=adata1.raw.X,index=adata1.obs.index,columns=adata1.var.index)
df_p['leiden'] = adata1.obs['leiden']
g = sns.clustermap(df_p.groupby('leiden').mean().rename({'eccentricity':'eccen.'},axis=1).rename(d_cluster, axis=0),
                   z_score=1,figsize=(6.2,6),cmap='viridis',
                   vmin=-2,vmax=2,cbar_pos=(.05, .89, .10, .05),cbar_kws={'orientation': 'horizontal','label':'Z-score'}) #(left, bottom, width, height),
g.savefig(f'./{s_date}/clustermap_leiden_{resolution}_{n_markers}.pdf',dpi=300)
g.savefig(f'./{s_date}/clustermap_leiden_{resolution}_{n_markers}.png',dpi=300)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
# stacked bar vertical

df = pd.DataFrame(data=adata1.raw.X,index=adata1.obs.index,columns=adata1.var.index)
df[f'leiden'] = [int(item) for item in adata1.obs.leiden]
s_markers = n_markers
k=resolution

df['slide'] = [item.split('_')[0] for item in df.index]
df['slide_scene'] = [item.split('_cell')[0] for item in df.index]
df['slide_scene'] = df.slide_scene.replace({'SMTBx2-5_scene001':'Bx2', 'SMTBx2-5_scene002':'Bx2',
       'SMTBx3_scene004':'Bx3', 'SMTBx4-3_scene001':'Bx4',
       'SMTBx4-3_scene002':'Bx4'})#.replace(d_order)
df['scene'] = [item.split('_')[1] for item in df.index]
df_prop = (df.groupby([f'leiden','slide_scene']).PCNA.count())/(df.groupby(['slide_scene']).PCNA.count())
df_prop = df_prop.unstack().fillna(value=0).T

fig,ax=plt.subplots(figsize=(5,6), dpi=200)
df_prop['slide'] =[item.split('_')[0] for item in df_prop.index]
#df_prop['scene'] =[item.split('_')[1] for item in df_prop.index]
df_prop = df_prop.loc[ls_order_r]
df_prop.columns = [str(item) for item in df_prop.columns]
#df_prop.rename(d_order).rename(d_cluster,axis=1).plot(kind='barh',stacked=True,ax=ax,legend=True,cmap='tab20',width=0.9)
df_prop.plot(kind='barh',stacked=True,ax=ax,legend=True,cmap='tab20',width=0.9)
ax.legend(bbox_to_anchor=(1.05, 1.00),ncol=1, fancybox=True,title='Cluster ID')
ax.set_xlabel('Fraction of Cells')
ax.set_ylabel('Tissue')
ax.set_title('')
plt.tight_layout()
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_vertical.pdf')
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_vertical.png')
#plt.close()


In [ ]:
#save the cluster ID, not hte annotation
#df_prop.to_csv(f'{s_sample}_{n_markers}markers_leiden{resolution}_frac_pos.csv')

In [ ]:
import matplotlib.ticker as tic
#SMT
fig,ax=plt.subplots(figsize=(2.8,3.2),dpi=200)
df_plot = df_prop.loc[['Bx2','Bx3','Bx4'],df_prop.dtypes=='float64'].T[::-1]
df_plot.plot(kind='barh',ax=ax,legend=True,width=.9)
ax.legend(title='Bx', loc='upper left',fancybox=True,borderpad=.2,bbox_to_anchor=(1.05, 1.05))
ax.set_xlabel('Fraction of Cells')
ax.set_ylabel('')
fig.suptitle(f'Cluster Composition: Biopsies',x=.5, y=.92)
for tick in ax.yaxis.get_major_ticks():
    tick.tick1line.set_markersize(0)
    tick.tick2line.set_markersize(0)
temp = tic.LinearLocator(numticks=18)
ax.yaxis.set_minor_locator(temp)
plt.grid(b=True, which='minor', axis='y')
plt.tight_layout()
fig.savefig(f'./{s_date}/Barplot_SMT{s_markers}_K{k}.pdf')
fig.savefig(f'./{s_date}/Barplot_SMT{s_markers}_K{k}.png')


In [ ]:
ls_order = ['Bx2', 'Bx3', 'Bx4','AU565-2','AU565-3', 'AU565-4', 'BT474-2','BT474-3', 'BT474-4', 
       'HCC1143-2', 'HCC1143-4', 'HCC3153-2', 'HCC3153-4', #'JE-TMA-43_scene01','JE-TMA-62_scene01', 'JE-TMA-43_scene10',
        'MDAMB-436-2','MDAMB-436-3', 'MDAMB-436-4', 'T47D-2','T47D-3', 'T47D-4',
       'N.Breast-2', 'N.Breast-4', 'tonsil-2', 'tonsil-4']
d_order = {#'
       'JE-TMA-43_scene02':'HCC1143-2', 'JE-TMA-62_scene02':'HCC1143-4',
       'JE-TMA-43_scene03':'HCC3153-2', 'JE-TMA-62_scene03':'HCC3153-4', 'JE-TMA-43_scene04':'N.Breast-2',
       'JE-TMA-62_scene04':'N.Breast-4', 'JE-TMA-43_scene05':'T47D-2', 'JE-TMA-62_scene05':'T47D-4',
       'JE-TMA-43_scene06':'T47D-2', 'JE-TMA-62_scene06':'T47D-4', 'JE-TMA-43_scene07':'tonsil-2',
       'JE-TMA-62_scene07':'tonsil-4', 'JE-TMA-43_scene08':'BT474-2', 'JE-TMA-62_scene08':'BT474-4',
       'JE-TMA-43_scene09':'BT474-2', 'JE-TMA-62_scene09':'BT474-4',  'JE-TMA-43_scene10':'AU565-2','JE-TMA-62_scene10':'AU565-4',
       'JE-TMA-43_scene11':'AU565-2', 'JE-TMA-62_scene11':'AU565-4', 'JE-TMA-43_scene13':'MDAMB-436-2',
       'JE-TMA-62_scene12':'MDAMB-436-4', 'JE-TMA-43_scene14':'MDAMB-436-2', 'JE-TMA-62_scene13':'MDAMB-436-4',
       'JE-TMA-60_scene13':'MDAMB-436-3', 'JE-TMA-60_scene11':'AU565-3', 'JE-TMA-60_scene10':'AU565-3',
       'JE-TMA-60_scene09':'BT474-3', 'JE-TMA-60_scene08':'BT474-3', 'JE-TMA-60_scene06':'T47D-3'}

In [ ]:
#stacked bar vertical tissue
df['coreID'] = df.slide_scene.replace(d_order)
df['celltype'] = df.leiden.astype('str').replace(d_clust_names)
df_prop = (df.groupby([f'celltype','coreID']).PCNA.count())/(df.groupby(['coreID']).PCNA.count())
df_prop = df_prop.unstack().fillna(value=0).T

fig,ax=plt.subplots(figsize=(5,3.7), dpi=200)
df_prop['slide'] =[item.split('_')[0] for item in df_prop.index]
ls_order_r = ls_order[::-1]
df_prop = df_prop.loc[ls_order_r]
df_prop.columns = [str(item) for item in df_prop.columns]
df_prop.plot(kind='barh',stacked=True,ax=ax,legend=True,cmap='tab20',width=0.9) #.rename(d_order).rename(d_clust_names,axis=1)
ax.legend(loc='upper left', bbox_to_anchor=(1.1,1.02),ncol=1, fancybox=True,title='Cluster Annotation')
ax.set_xlabel('Fraction of Cells')
ax.set_ylabel('Tissue')
ax.set_title('Cluster Composition: Biopsies Plus Controls')
plt.tight_layout()
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_withcontrols_vert.pdf')
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_withcontrols_vert.png')

In [ ]:
#stacked bar horizontal
df['coreID'] = df.slide_scene.replace(d_order)
df['celltype'] = df.leiden.astype('str').replace(d_clust_names)
df_prop = (df.groupby([f'celltype','coreID']).PCNA.count())/(df.groupby(['coreID']).PCNA.count())
df_prop = df_prop.unstack().fillna(value=0).T

fig,ax=plt.subplots(figsize=(10,2.5), dpi=200)
df_prop['slide'] =[item.split('_')[0] for item in df_prop.index]
#df_prop['scene'] =[item.split('_')[1] for item in df_prop.index]
df_prop = df_prop.loc[ls_order]
df_prop.columns = [str(item) for item in df_prop.columns]
df_prop.plot(kind='bar',stacked=True,ax=ax,legend=True,cmap='tab20',width=0.9) #.rename(d_order).rename(d_clust_names,axis=1)
ax.legend(loc='upper center', bbox_to_anchor=(1.5, 1.05),ncol=2, fancybox=True,title='Cluster Annotation')
ax.set_ylabel('Fraction of Cells')
ax.set_xlabel('Tissue')
ax.set_title('')
plt.tight_layout()
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_withcontrols.pdf')
fig.savefig(f'./{s_date}/StackedBar_{s_markers}markers_{k}Clusters_withcontrols.png')

In [ ]:
#plot all groups spatially  
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
newcmap = ListedColormap(mpl.cm.tab20.colors)#ListedColormap(mpl.cm.tab20b.colors + mpl.cm.tab20c.colors)
from mplex_image import analyze
df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_xy = df_mi.loc[df_pos.index]
ls_scene = ['SMTBx2-5_scene001', 'SMTBx3_scene004', 'SMTBx4-3_scene001', 'SMTBx4-3_scene002']
#ls_scene = ['JE-TMA-62_scene04', 'JE-TMA-43_scene04','JE-TMA-62_scene07','JE-TMA-43_scene07']
for s_slide in ls_scene:
    fig,ax = plt.subplots(figsize=(10,10),dpi=200) #10,10
    #plot negative cells
    df_scene = df_xy[df_xy.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
        s_color = str(s_color_int)
        if len(df_xy[(df_xy.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ax.scatter(data=df_xy[(df_xy.slide_scene==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                                                                        label=f'{s_color}',s=0.1,color=newcmap.colors[idxs])
        #break
    ax.set_title(f"{s_slide}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10) 
    fig.savefig(f'{codedir}/paper_data/GatingPlots/{s_slide}_clustering_scatterplot.png')
    #break

In [ ]:
if not os.path.exists(f'{s_sample}_{n_markers}markers_leiden{resolution}.csv'):
    print('saving csv')
    df.to_csv(f'{s_sample}_{n_markers}markers_leiden{resolution}.csv')
    df_prop.to_csv(f'{s_sample}_{n_markers}markers_leiden{resolution}_frac_pos.csv')

In [ ]:
f'{s_sample}_{n_markers}markers_leiden{resolution}.csv'

In [ ]:
f'{s_sample}_{n_markers}markers_leiden{resolution}_frac_pos.csv'